In [1]:
import os
import pandas as pd
import numpy as np
import bitsandbytes as bnb
import torch
from torch import nn
from datasets import load_dataset, Dataset
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import classification_report, accuracy_score

2025-04-05 21:06:01.993370: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import subprocess
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
model_path = "/root/autodl-tmp/7102_llama/LLM-Research/Meta-Llama-3-8B-Instruct/"

In [4]:
df = pd.read_csv("Combined Data.csv")
df.dropna(subset = ["statement", "status"], inplace = True)
df.reset_index(drop = True, inplace = True)

labels = sorted(df['status'].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}
df['label'] = df['status'].map(label2id)

In [5]:
train_df = df.sample(frac = 0.9, random_state = 42)
val_df = df.drop(train_df.index)

train_dataset = Dataset.from_pandas(train_df[["statement", "label"]])
val_dataset = Dataset.from_pandas(val_df[["statement", "label"]])

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code = True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.add_special_tokens({'pad_token': tokenizer.pad_token})
    tokenizer.save_pretrained("./tokenizer_with_pad")
    tokenizer = AutoTokenizer.from_pretrained("./tokenizer_with_pad", trust_remote_code=True)

In [7]:
def tokenize_function(example):
    return tokenizer(
        example["statement"],
        truncation = True,
        padding = "max_length",
        max_length = 256
    )

In [8]:
train_dataset = train_dataset.map(tokenize_function, batched = True)
val_dataset = val_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/47413 [00:00<?, ? examples/s]

Map:   0%|          | 0/5268 [00:00<?, ? examples/s]

In [9]:
train_dataset.set_format(type = "torch", columns = ["input_ids", "attention_mask", "label"])
val_dataset.set_format(type = "torch", columns = ["input_ids", "attention_mask", "label"])

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [11]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(label2id),
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/7102_llama/LLM-Research/Meta-Llama-3-8B-Instruct/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
base_model.resize_token_embeddings(len(tokenizer))
base_model.config.pad_token_id = tokenizer.pad_token_id

In [13]:
print("Pad token:", tokenizer.pad_token)
print("Pad token ID:", tokenizer.pad_token_id)

Pad token: <|eot_id|>
Pad token ID: 128009


In [14]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

In [15]:
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()  # 显示可训练参数数量

trainable params: 21,000,192 || all params: 7,525,953,536 || trainable%: 0.2790


In [16]:
print(model.config.pad_token_id)

128009


In [17]:
training_args = TrainingArguments(
    output_dir="./llama3_lora_output",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

/root/miniconda3/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    try:
        print(classification_report(labels, preds, target_names = labels.tolist()))
    except:
        print(classification_report(labels, preds))
    return {"accuracy": acc}

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

/tmp/ipykernel_10153/3437368569.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.450900,0.418408,0.828018
1,0.315200,0.523914,0.853834


              precision    recall  f1-score   support

           0       0.83      0.95      0.88       381
           1       0.86      0.90      0.88       283
           2       0.76      0.80      0.78      1537
           3       0.95      0.96      0.95      1600
           4       0.84      0.73      0.78       109
           5       0.77      0.79      0.78       280
           6       0.74      0.64      0.69      1078

    accuracy                           0.83      5268
   macro avg       0.82      0.82      0.82      5268
weighted avg       0.83      0.83      0.83      5268



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [23]:
trainer.save_model("./llama3_lora_classifier")
tokenizer.save_pretrained("./llama3_lora_classifier")

('./llama3_lora_classifier/tokenizer_config.json',
 './llama3_lora_classifier/special_tokens_map.json',
 './llama3_lora_classifier/tokenizer.json')

In [24]:
def predict(text):
    model.eval()
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        return id2label[pred]

In [25]:
print(predict("I am feeling extremely anxious today."))
print(predict("Life has been good lately. I feel happy."))

Anxiety
Normal


In [27]:
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       381
           1       0.95      0.89      0.92       283
           2       0.80      0.78      0.79      1537
           3       0.96      0.97      0.97      1600
           4       0.83      0.81      0.82       109
           5       0.83      0.85      0.84       280
           6       0.73      0.76      0.74      1078

    accuracy                           0.86      5268
   macro avg       0.86      0.86      0.86      5268
weighted avg       0.86      0.86      0.86      5268



In [29]:
print("📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(label2id.keys())))

📊 Classification Report:
                      precision    recall  f1-score   support

             Anxiety       0.94      0.93      0.94       381
             Bipolar       0.95      0.89      0.92       283
          Depression       0.80      0.78      0.79      1537
              Normal       0.96      0.97      0.97      1600
Personality disorder       0.83      0.81      0.82       109
              Stress       0.83      0.85      0.84       280
            Suicidal       0.73      0.76      0.74      1078

            accuracy                           0.86      5268
           macro avg       0.86      0.86      0.86      5268
        weighted avg       0.86      0.86      0.86      5268

